2024-02-19 13:34:20.487917: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 13:34:20.487944: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 13:34:20.489110: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 13:34:20.495489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-19 13:34:21.189766: W tensorflow/compiler/tf2

In [1]:
import tensorflow as tf
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

from utils.helper import *
from utils.constants import *
from inception.main import build_model

df = read_dataset("ArrowHead")

X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)
# apply one-hot encoder
y_train=to_categorical(df[1])
y_test=to_categorical(df[3])

model = build_model(input_shape=X_train.shape[1:] , num_classes=y_train.shape[1], num_modules=1)
model.summary()

2024-02-19 14:45:57.557617: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 14:45:57.557656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 14:45:57.558654: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 14:45:57.564340: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-19 14:45:58.300156: W tensorflow/compiler/tf2

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 251, 1)]             0         []                            
                                                                                                  
 conv1d (Conv1D)             (None, 251, 32)              32        ['input_1[0][0]']             
                                                                                                  
 max_pooling1d (MaxPooling1  (None, 251, 1)               0         ['input_1[0][0]']             
 D)                                                                                               
                                                                                                  
 conv1d_1 (Conv1D)           (None, 251, 32)              10240     ['conv1d[0][0]']          

2024-02-19 14:45:59.299741: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-19 14:45:59.334114: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-19 14:45:59.334292: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [7]:
num_modules=5
results=[]
histories=[]

try:
    for dataset in  ['ArrowHead', 'Wine', 'OliveOil', 'Car', 'BeetleFly']:
        with tf.device("/device:GPU:0"):
            # Load data
            df = read_dataset(dataset)

            X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
            X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)

            # apply one-hot encoder
            y_train=to_categorical(df[1])
            y_test=to_categorical(df[3])

            # Compile the model
            model = build_model(input_shape=X_train.shape[1:] , num_classes=y_train.shape[1], num_modules=num_modules)

            callbacks =[
                tf.keras.callbacks.ModelCheckpoint(
                    f"best_models/best_model_{dataset}.tf", save_weights_only=True, monitor="lr"
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor="lr", factor=0.5, patience=50, min_lr=0.0001
                ),
            ]

            # Train the model
            history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data= (df[2], y_test), callbacks = callbacks)

            # Evaluate the model
            train_loss, train_accuracy = model.evaluate(X_train, y_train)
            test_loss, test_accuracy = model.evaluate(X_test, y_test)

            results.append([dataset, train_loss, train_accuracy, test_loss, test_accuracy])
            histories.append(history.history)
            
    pd.DataFrame(results, columns=["Dataset","Train loss", "Train accuracy","Test loss", "Test accuracy"]).to_csv(f"result_{num_modules}.csv", index=False)

except Exception as e:
    print("An error occurred:", e)


for i in range(len(histories)):
    plot_loss(histories[i], UNIVARIATE_DATASET_NAMES_2018[i], num_modules)

Epoch 1/1000
3/3 [==============================] - 6s 394ms/step - loss: 1.4283 - accuracy: 0.3611 - val_loss: 114.3001 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 2/1000
3/3 [==============================] - 1s 235ms/step - loss: 1.0941 - accuracy: 0.5556 - val_loss: 575.7634 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 3/1000
3/3 [==============================] - 1s 233ms/step - loss: 0.7612 - accuracy: 0.6111 - val_loss: 1272.0709 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 4/1000
3/3 [==============================] - 1s 227ms/step - loss: 0.6294 - accuracy: 0.7222 - val_loss: 1501.7515 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 5/1000
3/3 [==============================] - 1s 232ms/step - loss: 0.5857 - accuracy: 0.7778 - val_loss: 1144.0966 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 6/1000
3/3 [==============================] - 1s 231ms/step - loss: 0.6599 - accuracy: 0.6944 - val_loss: 826.2023 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 7/1000
3/3 [==============================] - 1s 

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [8]:
num_modules=6
results=[]
histories=[]

try:
    for dataset in  ['ArrowHead', 'Wine', 'OliveOil', 'Car', 'BeetleFly']:
        with tf.device("/device:GPU:0"):
            # Load data
            df = read_dataset(dataset)

            X_train = df[0].reshape(df[0].shape[0], df[0].shape[1], 1)
            X_test = df[2].reshape(df[2].shape[0], df[2].shape[1], 1)

            # apply one-hot encoder
            y_train=to_categorical(df[1])
            y_test=to_categorical(df[3])

            # Compile the model
            model = build_model(input_shape=X_train.shape[1:] , num_classes=y_train.shape[1], num_modules=num_modules)

            callbacks =[
                tf.keras.callbacks.ModelCheckpoint(
                    f"best_models/best_model_{dataset}.tf", save_weights_only=True, monitor="lr"
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor="lr", factor=0.5, patience=50, min_lr=0.0001
                ),
            ]

            # Train the model
            history = model.fit(X_train, y_train, batch_size=16, epochs=1000, validation_data= (df[2], y_test), callbacks = callbacks)

            # Evaluate the model
            train_loss, train_accuracy = model.evaluate(X_train, y_train)
            test_loss, test_accuracy = model.evaluate(X_test, y_test)

            results.append([dataset, train_loss, train_accuracy, test_loss, test_accuracy])
            histories.append(history.history)
            
    pd.DataFrame(results, columns=["Dataset","Train loss", "Train accuracy","Test loss", "Test accuracy"]).to_csv(f"result_{num_modules}.csv", index=False)

except Exception as e:
    print("An error occurred:", e)


for i in range(len(histories)):
    plot_loss(histories[i], UNIVARIATE_DATASET_NAMES_2018[i], num_modules)

Epoch 1/1000
3/3 [==============================] - 8s 845ms/step - loss: 1.2985 - accuracy: 0.3333 - val_loss: 422.5480 - val_accuracy: 0.3029 - lr: 0.0100
Epoch 2/1000
3/3 [==============================] - 1s 293ms/step - loss: 0.8962 - accuracy: 0.5556 - val_loss: 7846.4141 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 3/1000
3/3 [==============================] - 1s 289ms/step - loss: 0.7022 - accuracy: 0.6667 - val_loss: 11191.1426 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 4/1000
3/3 [==============================] - 1s 283ms/step - loss: 0.6201 - accuracy: 0.8056 - val_loss: 10225.0596 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 5/1000
3/3 [==============================] - 1s 275ms/step - loss: 0.6110 - accuracy: 0.7222 - val_loss: 8440.8691 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 6/1000
3/3 [==============================] - 1s 281ms/step - loss: 0.5746 - accuracy: 0.8333 - val_loss: 5440.4077 - val_accuracy: 0.3943 - lr: 0.0100
Epoch 7/1000
3/3 [==============================] -

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>